# 이미지의 강도를 바꿀 수 있게 수정

In [46]:
#이미지 업로드
#파일 이름 변경할 것

from google.colab import files

uploaded = files.upload() 
for name, data in uploaded.items():
  
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)

Saving content_2.jpg to content_2.jpg
saved file content_2.jpg


In [47]:
from keras.preprocessing.image import load_img, img_to_array, save_img

target_image_path = 'content_2.jpg'
style_reference_image_path = 'style.jpg'

# 생성된 사진의 차원
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [48]:
#유틸리티 함수 정의

import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
  img = load_img(image_path, target_size = (img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x, avg_color):
  x[:, :, 0] += avg_color[0]
  x[:, :, 1] += avg_color[1]
  x[:, :, 2] += avg_color[2]

  x = x[:, :, ::-1]
  x = np.clip(x, 0, 255).astype('uint8') #0보다 작은 값을 0으로 대체
  return x

In [49]:
# 8-16 훈련된 VGG19 네트워크를 로딩하고 3개 이미지에 적용하기

from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis = 0)

model = vgg19.VGG19(input_tensor = input_tensor, weights = 'imagenet', include_top = False)


In [50]:
#8-17 콘텐츠 손실
def content_loss(base, combination):
  return K.sum(K.square(combination -  base))

In [51]:
#8-18 스타일 손실
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
  gram = K.dot(features, K.transpose(features))
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C)) / (4. * (channels **2) *(size **2))

In [52]:
#8-19 #총 변위 손실

def total_variation_loss(x):
  a = K.square(
      x[:, :img_height-1, :img_width-1, :] -
      x[:, 1:, :img_width-1, :]
  )

  b = K.square(
      x[:, :img_height - 1, :img_width - 1, :] -
      x[:, :img_height -1, 1:, :]
  )
  return K.sum(K.pow(a + b, 1.25))

In [53]:
# 8-20 최소화할 최종 손실 정의하기
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

content_layer = 'block5_conv2'
style_layers = [
                'block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

total_variation_weight = 0.001
style_weight = 0.25
content_weight = 0.001

loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

loss = loss + content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss = loss + (style_weight / len(style_layers)) * sl

loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [54]:
grads = K.gradients(loss, combination_image)[0]

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
  
  def __init__(self):
    self.loss_value = None
    self.grads_values = None

  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value

  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_value = None

    return grad_values

evaluator = Evaluator()

In [55]:
#코드 8-22 스타일 트랜스퍼 반복 루프

from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'style_transfer_result'
iterations = 20

x = preprocess_image(target_image_path)
myimg = cv2.imread(target_image_path)
avg_color_per_row = numpy.average(myimg, axis=0)
avg_color = numpy.average(avg_color_per_row, axis=0)

x = x.flatten()

for i in range(iterations):
  print('반복 횟수', i)
  start_time = time.time()
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime = evaluator.grads, maxfun = 20)
  
  print('현제 손실 값: ', min_val)
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img, avg_color)
  fname = result_prefix + '_at_iteration_{}.png' .format(i)
  save_img(fname, img)
  print('이미지 저장: ', fname)
  end_time = time.time()
  print('{} 번째 반복 완료: {}s' .format(i, end_time - start_time))

반복 횟수 0
현제 손실 값:  995613950.0
이미지 저장:  style_transfer_result_at_iteration_0.png
0 번째 반복 완료: 3.0295422077178955s
반복 횟수 1
현제 손실 값:  332177630.0
이미지 저장:  style_transfer_result_at_iteration_1.png
1 번째 반복 완료: 2.431666851043701s
반복 횟수 2
현제 손실 값:  188414820.0
이미지 저장:  style_transfer_result_at_iteration_2.png
2 번째 반복 완료: 2.443136692047119s
반복 횟수 3
현제 손실 값:  132939710.0
이미지 저장:  style_transfer_result_at_iteration_3.png
3 번째 반복 완료: 2.426894426345825s
반복 횟수 4
현제 손실 값:  110321410.0
이미지 저장:  style_transfer_result_at_iteration_4.png
4 번째 반복 완료: 2.4051172733306885s
반복 횟수 5
현제 손실 값:  91594720.0
이미지 저장:  style_transfer_result_at_iteration_5.png
5 번째 반복 완료: 2.4714763164520264s
반복 횟수 6
현제 손실 값:  79648320.0
이미지 저장:  style_transfer_result_at_iteration_6.png
6 번째 반복 완료: 2.5024850368499756s
반복 횟수 7
현제 손실 값:  71011830.0
이미지 저장:  style_transfer_result_at_iteration_7.png
7 번째 반복 완료: 2.4505906105041504s
반복 횟수 8
현제 손실 값:  65539810.0
이미지 저장:  style_transfer_result_at_iteration_8.png
8 번째 반복 완료: 2.4277443885803223s